In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

from nltk.tokenize import word_tokenize

import warnings
warnings.filterwarnings('ignore')

df = pd. read_csv('./Cleaned_Data.csv')
df.dropna(inplace=True)
df.head()

,id,label,tweet
0,1,0,user father dysfunctional selfish kids dysfunc...
1,2,0,user thanks lyft credit cause offer wheelchair...
2,3,0,majesty
3,4,0,model take time
4,5,0,factsguide society motivation


In [3]:
X = df['tweet']
Y = df['label']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1)

## Creating a Word2Vec model using gensim library

In [4]:
import gensim
from gensim.models.word2vec import Word2Vec

model = Word2Vec(size=100 , min_count=10)
model.build_vocab([x.strip(' ').split() for x in X_train])
model.train([x.strip(' ').split() for x in X_train], total_examples=model.corpus_count, epochs=model.epochs)

(468332, 797755)

In [15]:
model.most_similar('trump')

[('racist', 0.9895153045654297),
 ('obama', 0.982775866985321),
 ('comments', 0.9800493717193604),
 ('black', 0.9790303707122803),
 ('fuhered', 0.9789710640907288),
 ('canada', 0.9786760807037354),
 ('women', 0.9785375595092773),
 ('meets', 0.9783369302749634),
 ('snapshot', 0.9778761267662048),
 ('porn', 0.9766402244567871)]

### Saving the Word2vec model so as to load it again when required

In [6]:
model.save('tweets_word2vec.model')

In [7]:
Total_tweets_list = []
for tweet in X_train:
    tweet_words = word_tokenize(tweet)
    Total_tweets_list.append(tweet_words)

In [11]:
import math
def tf_idf(words, word):
    tf_count = 0.0
    idf_count = 0.0
    for i in words:
        if i == word:
            tf_count += 1
    for tweet_words in Total_tweets_list:
        if word in tweet_words:
            idf_count += 1
    return tf/len(words), math.log10(idf/len(Total_tweets_list));

## Creating a Input Dataframe using TF-IDF to create a classification Model

In [ ]:
input_dataframe = pd.DataFrame()
for tweet in X_train:
    words = tweet.split()
    final_word_vector = [0 for i in range(100)]
    for word in words:
        try:
            word_vector = (model['{}'.format(word)])
            tf,idf = tf_idf(words, word)
            word_vector = tf * idf * word_vector
            final_word_vector = final_word_vector + word_vector
        except:
            pass
    input_dataframe.append(pd.Series(final_word_vector), inplace = True)